In [147]:
!pip install --upgrade tensorflow-gpu==2.0

Requirement already up-to-date: tensorflow-gpu==2.0 in /usr/local/lib/python3.7/dist-packages (2.0.0)


In [148]:
!pip install plotly
!pip install --upgrade nbformat
!pip install nltk
!pip install spacy
!pip install WordCloud
!pip install gensim 
import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model



Requirement already up-to-date: nbformat in /usr/local/lib/python3.7/dist-packages (5.1.3)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [149]:
# load the data
import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

# create folder to write data to
B11=os.path.join(DRIVE_MOUNT, 'My Drive', 'B11_2021')
HOMEWORK_FOLDER=os.path.join(B11, 'Project')
os.makedirs(HOMEWORK_FOLDER, exist_ok=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [150]:
fake = pd.read_csv('/content/gdrive/MyDrive/B11_2021/Project/Fake.csv')
true = pd.read_csv('/content/gdrive/My Drive/B11_2021/Project/True.csv')

In [151]:
fake.drop("subject", axis=1, inplace=True)
true.drop("subject", axis=1, inplace=True)

In [152]:
true = true[true["text"] != ""] 
fake = fake[fake["text"] != ""] 

In [153]:
fake.drop("date", axis=1, inplace=True)
true.drop("date", axis=1, inplace=True)

In [154]:
fake.drop("title", axis=1, inplace=True)
true.drop("title", axis=1, inplace=True)

In [156]:
fake.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [157]:
final.shape

(2000, 4)

In [158]:
# add a target class column to indicate whether the news is real or fake
true['label'] = 1
fake['label'] = 0

In [159]:
final = pd.concat([true,fake], axis=0, ignore_index=True)

In [160]:
final.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [161]:
final['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [162]:
# download stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [163]:
# Obtain additional stopwords from nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [164]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [165]:
# Apply the function to the dataframe
final['clean'] = final['text'].apply(preprocess)

In [166]:
# Show original news
final['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [167]:
# Show cleaned up news after removing stopwords
print(final['clean'][0])

['washington', 'reuters', 'head', 'conservative', 'republican', 'faction', 'congress', 'voted', 'month', 'huge', 'expansion', 'national', 'debt', 'cuts', 'called', 'fiscal', 'conservative', 'sunday', 'urged', 'budget', 'restraint', 'keeping', 'sharp', 'pivot', 'republicans', 'representative', 'mark', 'meadows', 'speaking', 'face', 'nation', 'drew', 'hard', 'line', 'federal', 'spending', 'lawmakers', 'bracing', 'battle', 'january', 'return', 'holidays', 'wednesday', 'lawmakers', 'begin', 'trying', 'pass', 'federal', 'budget', 'fight', 'likely', 'linked', 'issues', 'immigration', 'policy', 'november', 'congressional', 'election', 'campaigns', 'approach', 'republicans', 'seek', 'control', 'congress', 'president', 'donald', 'trump', 'republicans', 'want', 'budget', 'increase', 'military', 'spending', 'democrats', 'want', 'proportional', 'increases', 'defense', 'discretionary', 'spending', 'programs', 'support', 'education', 'scientific', 'research', 'infrastructure', 'public', 'health', 'e

In [168]:
# Obtain the total words present in the dataset
list_of_words = []
for i in final.clean:
    for j in i:
        list_of_words.append(j)


In [171]:
# Obtain the total number of unique words
total_words = len(list(set(list_of_words)))
total_words

108259

In [172]:
# join the words into a string
final['clean_joined'] = final['clean'].apply(lambda x: " ".join(x))

In [173]:
final

,text,label,clean,clean_joined
0,WASHINGTON (Reuters) - The head of a conservat...,1,"[washington, reuters, head, conservative, repu...",washington reuters head conservative republica...
1,WASHINGTON (Reuters) - Transgender people will...,1,"[washington, reuters, transgender, people, all...",washington reuters transgender people allowed ...
2,WASHINGTON (Reuters) - The special counsel inv...,1,"[washington, reuters, special, counsel, invest...",washington reuters special counsel investigati...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1,"[washington, reuters, trump, campaign, adviser...",washington reuters trump campaign adviser geor...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1,"[seattle, washington, reuters, president, dona...",seattle washington reuters president donald tr...
...,...,...,...,...
44893,21st Century Wire says As 21WIRE reported earl...,0,"[century, wire, says, wire, reported, earlier,...",century wire says wire reported earlier week u...
44894,21st Century Wire says It s a familiar theme. ...,0,"[century, wire, says, familiar, theme, dispute...",century wire says familiar theme dispute chang...
44895,Patrick Henningsen 21st Century WireRemember ...,0,"[patrick, henningsen, century, wireremember, o...",patrick henningsen century wireremember obama ...
44896,21st Century Wire says Al Jazeera America will...,0,"[century, wire, says, jazeera, america, histor...",century wire says jazeera america history bigg...


In [174]:
final['clean_joined'][0]

'washington reuters head conservative republican faction congress voted month huge expansion national debt cuts called fiscal conservative sunday urged budget restraint keeping sharp pivot republicans representative mark meadows speaking face nation drew hard line federal spending lawmakers bracing battle january return holidays wednesday lawmakers begin trying pass federal budget fight likely linked issues immigration policy november congressional election campaigns approach republicans seek control congress president donald trump republicans want budget increase military spending democrats want proportional increases defense discretionary spending programs support education scientific research infrastructure public health environmental protection trump administration willing going increase defense discretionary spending percent meadows chairman small influential house freedom caucus said program democrats saying need government raise percent fiscal conservative rationale eventually p

In [175]:
# length of maximum document will be needed to create word embeddings 
maxlen = -1
for doc in final.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)


The maximum number of words in any document is = 4399


In [176]:
# split data into test and train 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final.clean_joined, final.label, test_size = 0.2)

In [177]:
from nltk import word_tokenize

In [178]:

tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)


In [180]:

padded_train = pad_sequences(train_sequences,maxlen = 40, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 40, truncating = 'post') 

In [182]:

model = Sequential()
model.add(Embedding(total_words, output_dim = 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         13857152  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 14,153,345
Trainable params: 14,153,345
Non-trainable params: 0
_________________________________________________________________


In [183]:
total_words

108259

In [184]:
y_train = np.asarray(y_train)

In [185]:
# train the model
model.fit(padded_train, y_train, batch_size = 100, validation_split = 0.1, epochs = 2)

Train on 32326 samples, validate on 3592 samples
Epoch 1/2
32326/32326 [==============================] - 68s 2ms/sample - loss: 0.0536 - acc: 0.9860 - val_loss: 0.0031 - val_acc: 0.9997
Epoch 2/2
32326/32326 [==============================] - 62s 2ms/sample - loss: 0.0029 - acc: 0.9994 - val_loss: 0.0065 - val_acc: 0.9981


In [186]:
# make prediction
pred = model.predict(padded_test)

In [187]:
# if the predicted value is >0.5 it is real else it is fake
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [188]:
# getting the accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)

Model Accuracy :  0.9810690423162584
